From https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627

In [1]:
import numpy as np
import torch
import torchvision
# import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [3]:
trainset = datasets.MNIST('data/', download=True, train=True, transform=transform)
valset = datasets.MNIST('data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [4]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [7]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
print(images.shape)
images = images.view(images.shape[0], -1)
print(images.shape)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

torch.Size([64, 1, 28, 28])
torch.Size([64, 784])


In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.6453375548426149
Epoch 1 - Training loss: 0.2839471120427031
Epoch 2 - Training loss: 0.2227434718858268
Epoch 3 - Training loss: 0.1791206600148477
Epoch 4 - Training loss: 0.1501937221699972
Epoch 5 - Training loss: 0.12870397312697698
Epoch 6 - Training loss: 0.11300322995769349
Epoch 7 - Training loss: 0.09975925623737117
Epoch 8 - Training loss: 0.08901361081260703
Epoch 9 - Training loss: 0.07943689126296957
Epoch 10 - Training loss: 0.07285331765702094
Epoch 11 - Training loss: 0.06656282975612435
Epoch 12 - Training loss: 0.06108577304712332
Epoch 13 - Training loss: 0.056353944680541314
Epoch 14 - Training loss: 0.051251063157351555

Training Time (in minutes) = 6.46910723845164


In [15]:
images, labels = next(iter(valloader))

img = images[0].view(1, 784)
with torch.no_grad():
    logps = model(img)
    print(logps)

ps = torch.exp(logps)
print(ps)
probab = list(ps.numpy()[0])
print(probab)
print("Predicted Digit =", probab.index(max(probab)))
# view_classify(img.view(1, 28, 28), ps)

tensor([[-1.5040e+01, -1.8708e+01, -1.3163e+01, -1.3056e+01, -1.7572e+01,
         -1.1867e+01, -1.0732e+01, -2.2547e+01, -3.3259e-05, -1.6541e+01]])
tensor([[2.9382e-07, 7.5022e-09, 1.9212e-06, 2.1375e-06, 2.3369e-08, 7.0159e-06,
         2.1845e-05, 1.6150e-10, 9.9997e-01, 6.5500e-08]])
[2.9381582e-07, 7.502205e-09, 1.9211984e-06, 2.1375322e-06, 2.336931e-08, 7.0159404e-06, 2.1845306e-05, 1.6149901e-10, 0.99996674, 6.5499734e-08]
Predicted Digit = 8


In [11]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9749


In [12]:
torch.save(model, 'data/models/mnist.pt') 

In [8]:
model = torch.load('data/models/mnist.pt')